In [73]:
#Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')

In [74]:
df = pd.read_csv('/content/sales_predictions.csv')

In [75]:
df.head()

Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
0           FDA15         9.30          Low Fat         0.016047   
1           DRC01         5.92          Regular         0.019278   
2           FDN15        17.50          Low Fat         0.016760   
3           FDX07        19.20          Regular         0.000000   
4           NCD19         8.93          Low Fat         0.000000   

               Item_Type  Item_MRP Outlet_Identifier  \
0                  Dairy  249.8092            OUT049   
1            Soft Drinks   48.2692            OUT018   
2                   Meat  141.6180            OUT049   
3  Fruits and Vegetables  182.0950            OUT010   
4              Household   53.8614            OUT013   

   Outlet_Establishment_Year Outlet_Size Outlet_Location_Type  \
0                       1999      Medium               Tier 1   
1                       2009      Medium               Tier 3   
2                       1999      Medium               Tier 1   
3                       1998         NaN               Tier 3   
4                       1987        High               Tier 3   

         Outlet_Type  Item_Outlet_Sales  
0  Supermarket Type1          3735.1380  
1  Supermarket Type2           443.4228  
2  Supermarket Type1          2097.2700  
3      Grocery Store           732.3800  
4  Supermarket Type1           994.7052

#• Getting rid of unnecessary columns

In [76]:
df = df.drop(columns = ['Item_Identifier','Outlet_Identifier','Item_Weight'])

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Fat_Content           8523 non-null   object 
 1   Item_Visibility            8523 non-null   float64
 2   Item_Type                  8523 non-null   object 
 3   Item_MRP                   8523 non-null   float64
 4   Outlet_Establishment_Year  8523 non-null   int64  
 5   Outlet_Size                6113 non-null   object 
 6   Outlet_Location_Type       8523 non-null   object 
 7   Outlet_Type                8523 non-null   object 
 8   Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 599.4+ KB


##I see categorical values that'll need to be imputed.

## Looking for duplicate values.

In [78]:
df.duplicated().sum()

0

In [79]:
columns = df.columns
columns

Index(['Item_Fat_Content', 'Item_Visibility', 'Item_Type', 'Item_MRP',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

#• Checking for questionable values. 

In [80]:
for col in columns:
    print(f'{col}:')
    print(df[col].describe())
    print('\n\n')

Item_Fat_Content:
count        8523
unique          5
top       Low Fat
freq         5089
Name: Item_Fat_Content, dtype: object



Item_Visibility:
count    8523.000000
mean        0.066132
std         0.051598
min         0.000000
25%         0.026989
50%         0.053931
75%         0.094585
max         0.328391
Name: Item_Visibility, dtype: float64



Item_Type:
count                      8523
unique                       16
top       Fruits and Vegetables
freq                       1232
Name: Item_Type, dtype: object



Item_MRP:
count    8523.000000
mean      140.992782
std        62.275067
min        31.290000
25%        93.826500
50%       143.012800
75%       185.643700
max       266.888400
Name: Item_MRP, dtype: float64



Outlet_Establishment_Year:
count    8523.000000
mean     1997.831867
std         8.371760
min      1985.000000
25%      1987.000000
50%      1999.000000
75%      2004.000000
max      2009.000000
Name: Outlet_Establishment_Year, dtype: float64



Outlet_Size:

##• No columns seem to contain any inconsistent or nonsensical values. And all seem to be the proper data type. 

Also, I see that I'll need 

#Validation Split

In [81]:
y = df.Item_Outlet_Sales
X = df.drop(columns = 'Item_Outlet_Sales')

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

#Instantiating column selectors

In [83]:
#Numeric:
num_selector = make_column_selector(dtype_include = 'number')
num_selector(X_train)

['Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year']

In [84]:
#Instantiating scaler and fitting it onto train data
scaler = StandardScaler()
scaler.fit_transform(X_train[num_selector(X_train)])

array([[-0.71277507,  1.82810922,  1.32784893],
       [-1.29105225,  0.60336888,  1.32784893],
       [ 1.81331864,  0.24454056,  0.13618724],
       ...,
       [-0.92052713,  1.52302674,  0.49368575],
       [-0.2277552 , -0.38377708,  1.0895166 ],
       [-0.95867683, -0.73836105, -0.10214509]])

In [85]:
cat_selector = make_column_selector(dtype_include = 'object')
cat_selector(X_train)

['Item_Fat_Content',
 'Item_Type',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [86]:
#Categorical data pipeline:
cat_imputer = SimpleImputer(strategy='most_frequent')
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
cat_pipe = make_pipeline(cat_imputer, ohe)
cat_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [87]:
#fitting pipelines to train data
cat_pipe.fit_transform(X_train[cat_selector(X_train)])

array([[0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.]])

# Making one large transformer from all column selectors, transformers, and the imputer. 

In [88]:
#letting make_column_tranformer take them in (pipeline, column selector) tuple format
preprocessor = make_column_transformer((cat_pipe, cat_selector),(scaler, num_selector))

In [89]:
preprocessor.fit_transform(X_train)

array([[ 0.        ,  1.        ,  0.        , ..., -0.71277507,
         1.82810922,  1.32784893],
       [ 0.        ,  0.        ,  1.        , ..., -1.29105225,
         0.60336888,  1.32784893],
       [ 0.        ,  0.        ,  1.        , ...,  1.81331864,
         0.24454056,  0.13618724],
       ...,
       [ 0.        ,  1.        ,  0.        , ..., -0.92052713,
         1.52302674,  0.49368575],
       [ 0.        ,  0.        ,  0.        , ..., -0.2277552 ,
        -0.38377708,  1.0895166 ],
       [ 1.        ,  0.        ,  0.        , ..., -0.95867683,
        -0.73836105, -0.10214509]])